In [8]:
import os
from os import path
import pandas as pd
import sys
basepath = path.join(os.getcwd(), "export_us")

In [9]:
import os
import numpy as np
import pandas as pd

start, end = '2020-06-30', '2024-06-27'
stock_codes = pd.read_csv('dji_companies.csv')['Symbol']

dfs = []
for stock_code in stock_codes:
    fname = f"{stock_code}.csv"
    fpath = os.path.join(basepath, fname)
    if os.path.exists(fpath):
        df = pd.read_csv(fpath, parse_dates=['日期'])
        df.rename(columns={'日期': 'date', '开盘': 'open', '收盘': 'close', '最高': 'high', '最低': 'low', '成交量': 'volume', '成交额': 'amount'}, inplace=True)
        df['stock_code'] = stock_code
        dfs.append(df[(df['date'] >= start) & (df['date'] <= end)])

all_df = pd.concat(dfs, ignore_index=True)
all_df.set_index(['stock_code', 'date'], inplace=True)
all_df.sort_index(inplace=True)

In [10]:
all_df

open   close    high     low    volume        amount  \
stock_code date                                                                 
AAPL       2020-06-30  85.385  86.565  86.860  85.365  35055821  1.275552e+10   
           2020-07-01  86.645  86.393  87.205  86.343  27684309  1.012571e+10   
           2020-07-02  87.328  86.393  87.983  86.275  28510367  1.046759e+10   
           2020-07-06  87.865  88.828  89.310  87.833  29663913  1.107689e+10   
           2020-07-07  89.218  88.538  90.020  88.423  28106114  1.056394e+10   
...                       ...     ...     ...     ...       ...           ...   
WMT        2024-02-08  55.160  55.157  55.550  55.080   4809309  8.159275e+08   
           2024-02-09  55.163  55.127  55.277  55.007   4776622  8.087225e+08   
           2024-02-12  55.110  55.467  55.487  55.050   4931680  8.379666e+08   
           2024-02-13  55.573  55.080  55.723  54.790   6191405  1.048497e+09   
           2024-02-14  54.950  54.900  54.992  54.547   6161534  1.036403e+09   

                         振幅   涨跌幅    涨跌额   换手率  
stock_code date                                 
AAPL       2020-06-30  1.74  0.88  0.755  0.82  
           2020-07-01  1.00 -0.20 -0.172  0.65  
           2020-07-02  1.98  0.00  0.000  0.67  
           2020-07-06  1.71  2.82  2.435  0.69  
           2020-07-07  1.80 -0.33 -0.290  0.66  
...                     ...   ...    ...   ...  
WMT        2024-02-08  0.85 -0.01 -0.003  0.18  
           2024-02-09  0.49 -0.05 -0.030  0.18  
           2024-02-12  0.79  0.62  0.340  0.18  
           2024-02-13  1.68 -0.70 -0.387  0.23  
           2024-02-14  0.81 -0.33 -0.180  0.23  

[27390 rows x 10 columns]

In [11]:
all_df

open   close    high     low    volume        amount  \
stock_code date                                                                 
AAPL       2020-06-30  85.385  86.565  86.860  85.365  35055821  1.275552e+10   
           2020-07-01  86.645  86.393  87.205  86.343  27684309  1.012571e+10   
           2020-07-02  87.328  86.393  87.983  86.275  28510367  1.046759e+10   
           2020-07-06  87.865  88.828  89.310  87.833  29663913  1.107689e+10   
           2020-07-07  89.218  88.538  90.020  88.423  28106114  1.056394e+10   
...                       ...     ...     ...     ...       ...           ...   
WMT        2024-02-08  55.160  55.157  55.550  55.080   4809309  8.159275e+08   
           2024-02-09  55.163  55.127  55.277  55.007   4776622  8.087225e+08   
           2024-02-12  55.110  55.467  55.487  55.050   4931680  8.379666e+08   
           2024-02-13  55.573  55.080  55.723  54.790   6191405  1.048497e+09   
           2024-02-14  54.950  54.900  54.992  54.547   6161534  1.036403e+09   

                         振幅   涨跌幅    涨跌额   换手率  
stock_code date                                 
AAPL       2020-06-30  1.74  0.88  0.755  0.82  
           2020-07-01  1.00 -0.20 -0.172  0.65  
           2020-07-02  1.98  0.00  0.000  0.67  
           2020-07-06  1.71  2.82  2.435  0.69  
           2020-07-07  1.80 -0.33 -0.290  0.66  
...                     ...   ...    ...   ...  
WMT        2024-02-08  0.85 -0.01 -0.003  0.18  
           2024-02-09  0.49 -0.05 -0.030  0.18  
           2024-02-12  0.79  0.62  0.340  0.18  
           2024-02-13  1.68 -0.70 -0.387  0.23  
           2024-02-14  0.81 -0.33 -0.180  0.23  

[27390 rows x 10 columns]

In [12]:
all_df.to_csv("all_stock_data.csv")

In [13]:
all_df_loaded = pd.read_csv("all_stock_data.csv", parse_dates=['date'])
all_df_loaded.set_index(['stock_code', 'date'], inplace=True)
all_df_loaded.sort_index(inplace=True)

In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import spearmanr
from scipy.stats import pearsonr


# 读取合并后的数据
all_df_loaded = pd.read_csv("all_stock_data.csv", parse_dates=['date'])
all_df_loaded.set_index(['stock_code', 'date'], inplace=True)
all_df_loaded.sort_index(inplace=True)


# 定义你的因子函数，输入为单个股票的DataFrame
def factor_func(df):
    # Calculate the difference between close and open prices, representing the price change over a day
    price_change = df['close'] - df['open']
    
    # Calculate typical price
    typical_price = (df['high'] + df['low'] + df['close']) / 3
    
    # Compute the ratio of volume to typical price as a measure of market interest
    volume_interest = df['volume'] / typical_price
    
    # Calculate the 10-day moving average of the typical price
    ma_10 = typical_price.rolling(window=10).mean()
    
    # Calculate the 5-day exponential moving average of the typical price
    ema_5 = typical_price.ewm(span=5, adjust=False).mean()
    
    # Calculate the Volume-Weighted Average Price (VWAP)
    vwap = (df['close'] * df['volume']).cumsum() / df['volume'].cumsum()
    
    # Adaptive window for volatility: calculate the standard deviation over an adaptive window
    window_size = 20
    volatility = typical_price.rolling(window=window_size).std()
    
    # Calculate the 14-day RSI
    delta = df['close'].diff(1)
    gain = (delta.where(delta > 0, 0)).fillna(0)
    loss = (-delta.where(delta < 0, 0)).fillna(0)
    avg_gain = gain.rolling(window=14).mean()
    avg_loss = loss.rolling(window=14).mean()
    rs = avg_gain / avg_loss
    rsi = 100 - (100 / (1 + rs))
    
    # Calculate the 20-day momentum
    momentum_20 = df['close'] - df['close'].shift(20)
    
    # Calculate the 50-day and 200-day moving averages of the close price
    ma_50 = df['close'].rolling(window=50).mean()
    ma_200 = df['close'].rolling(window=200).mean()
    
    # Calculate the 50-day and 200-day Bollinger Bands
    bb_50_middle = df['close'].rolling(window=50).mean()
    bb_50_std = df['close'].rolling(window=50).std()
    bb_50_upper = bb_50_middle + 2 * bb_50_std
    bb_50_lower = bb_50_middle - 2 * bb_50_std
    
    bb_200_middle = df['close'].rolling(window=200).mean()
    bb_200_std = df['close'].rolling(window=200).std()
    bb_200_upper = bb_200_middle + 2 * bb_200_std
    bb_200_lower = bb_200_middle - 2 * bb_200_std
    
    # Calculate the percentage of the close price relative to the 50-day and 200-day Bollinger Bands
    bb_50_percent = (df['close'] - bb_50_lower) / (bb_50_upper - bb_50_lower)
    bb_200_percent = (df['close'] - bb_200_lower) / (bb_200_upper - bb_200_lower)
    
    # Adaptive weights based on recent data and market conditions
    momentum_weight = 0.1 + 0.05 * (rsi - 50)
    mean_reversion_weight = 0.1 + 0.05 * (1 - (typical_price - ma_10) / volatility)
    trend_weight = 0.1 + 0.05 * ((ma_50 - ma_200) / volatility)
    
    # Create a weighted indicator using the price change, volume interest, and additional technical indicators
    alpha_factor = (
        0.4 * price_change 
        + 0.2 * volume_interest 
        + 0.1 * (ema_5 - ma_10) 
        + 0.1 * (vwap - typical_price) / volatility 
        + 0.1 * (rsi - 50) 
        + momentum_weight * momentum_20 
        + mean_reversion_weight * (ma_50 - ma_200) 
        + trend_weight * (bb_50_percent - bb_200_percent)
    )
    
    return alpha_factor

# 计算每只股票的因子值
all_df_loaded['factor'] = all_df_loaded.groupby('stock_code').apply(lambda g: factor_func(g)).reset_index(level=0, drop=True)

# 计算未来6日收益率
all_df_loaded['future_return_6d'] = all_df_loaded.groupby('stock_code')['close'].shift(-6) / all_df_loaded['close'] - 1

# 取所有日期
start_date = pd.Timestamp('2021-01-01')
end_date = pd.Timestamp('2024-01-01')
all_dates = all_df_loaded.index.get_level_values('date').unique()
all_dates = all_dates[(all_dates >= start_date) & (all_dates <= end_date)]
ic_values = []

for date in all_dates:
    daily = all_df_loaded.xs(date, level='date')
    factors = daily['factor']
    returns = daily['future_return_6d']
    mask = factors.notna() & returns.notna() & np.isfinite(factors) & np.isfinite(returns)
    if mask.sum() >= 10:
        ic, _ = pearsonr(factors[mask], returns[mask])
        if not np.isnan(ic):
            ic_values.append(ic)

mean_ic = np.mean(ic_values)
print(f"Mean IC: {mean_ic:.10f}")

Mean IC: 0.0483461432
